# SKLearn implementations of logistic regression and SVM
## This is done to compare the implementation of our models to the standard libraries
We fully expect that these results will be better than our results, especially for SVM. Our SVM implementation uses a simplified version of the learning algorithm.
The SKlearn package also does some intelligent processing and setting optimalisation underneath.

## Module imports

In [1]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.utils import resample

# SKlearn for F1 score calculation
from sklearn.metrics import f1_score

# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize

# we use pandas to import a comma-seperated values dataset
import pandas as pd

# will be used to load MATLAB mat datafile format
from scipy.io import loadmat

# library written for this exercise providing additional functions for assignment submission, and others
import svm

import logistic_regression as lr

# tells matplotlib to embed plots within the notebook
%matplotlib inline

## Reading the dataset and selecting features

In [2]:
data = pd.read_csv(os.path.join('data', 'healthcare-dataset-stroke-data.csv'))

data.head(10)

data.drop("id", axis = 1, inplace = True)
data.drop("gender", axis = 1, inplace = True)
#data.drop("ever_married", axis = 1, inplace = True)
data.drop("work_type", axis = 1, inplace = True)
#data.drop("Residence_type", axis = 1, inplace = True)
data.drop("bmi", axis = 1, inplace = True)
#data.drop("smoking_status", axis = 1, inplace = True)

#indexAge = data[((data['age'] < 20) & (data['stroke'] == 1)) ].index  #find the indexes of outliers
#data.drop(indexAge , inplace=True) # drop the outliers
#indexGlucose = data[((data['avg_glucose_level'] > 220) & (data['stroke'] == 0)) ].index
#data.drop(indexGlucose , inplace=True)

Removing all the empty datarows, only necessary when using the BMI feature.

In [3]:
data = data.dropna(axis = 0)

## Making dummy classes for the enumertor classes

In [6]:
num_cols = data.select_dtypes(include = [np.number]).columns.tolist()
obj_cols = data.select_dtypes(exclude = [np.number]).columns.tolist()

In [7]:
# Numerical columns data
data_new_num = data[num_cols]

# Categorical columns data
data_new_cat = data[obj_cols]

# Creating dummies
data_new_cat_dummies = pd.get_dummies(data_new_cat)
print(data_new_cat_dummies.shape)
data_new_cat_dummies.head()

(5110, 8)


,ever_married_No,ever_married_Yes,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,0,1,0,1,0,1,0,0
1,0,1,1,0,0,0,1,0
2,0,1,1,0,0,0,1,0
3,0,1,0,1,0,0,0,1
4,0,1,1,0,0,0,1,0


In [8]:
data_new_final = pd.concat([data_new_num, data_new_cat_dummies], axis = 1)
print(data_new_final.shape)
data_new_final.head()

(5110, 13)


,age,hypertension,heart_disease,avg_glucose_level,stroke,ever_married_No,ever_married_Yes,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,1,0,1,0,1,0,1,0,0
1,61.0,0,0,202.21,1,0,1,1,0,0,0,1,0
2,80.0,0,1,105.92,1,0,1,1,0,0,0,1,0
3,49.0,0,0,171.23,1,0,1,0,1,0,0,0,1
4,79.0,1,0,174.12,1,0,1,1,0,0,0,1,0


## Downsample and train test split

In [9]:
negative = data_new_final[data_new_final.stroke==0]
positive = data_new_final[data_new_final.stroke==1]

#print(data.stroke.value_counts())

#print(negative.stroke.value_counts())
#print(positive.stroke.value_counts())

# downsample majority
neg_downsampled = resample(negative,
 replace=True, # sample with replacement
 n_samples=len(positive), # match number in minority class
 random_state=27) # reproducible results
# combine minority and downsampled majority
downsampled = pd.concat([positive, neg_downsampled])
# check new class counts
downsampled.stroke.value_counts()

1    249
0    249
Name: stroke, dtype: int64

In [10]:
X = downsampled.drop(['stroke'], axis=1).values
y = downsampled['stroke'].values

In [11]:
# Setup the data matrix appropriately, and add ones for the intercept term
m, n = X.shape

print(X.shape)

# Add intercept term to X
X = np.concatenate([np.ones((m, 1)), X], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0,shuffle=True, stratify=y)

(498, 12)


## Implementation of the logistic regression

## Implenmentation of the SVM